# Ниязов Алдияр М.19-04 
Задание 1 Предсказание количества комментариев к посту в FaceBook

Произведем выгрузку данных из датасета

In [83]:
import pandas as pd
import numpy as np
data = pd.read_csv('Dataset/Training/Features_Variant_1.csv', header=None)

Посмотрим на них

In [84]:
data

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,0,0,0,1,0
1,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,0,0,1,0,0
2,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,0,0,0,0,0,0,1,0
3,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,0,1,0,0,0,0,0,0
4,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40944,7170111,70,497000,9,0.0,1881.0,497.200000,269.0,502.318385,0.0,...,0,0,0,0,0,0,0,1,0,1
40945,7170111,70,497000,9,0.0,1881.0,497.200000,269.0,502.318385,0.0,...,0,0,0,0,0,0,0,1,0,2
40946,7170111,70,497000,9,0.0,1881.0,497.200000,269.0,502.318385,0.0,...,0,0,0,0,0,0,1,0,0,72
40947,7170111,70,497000,9,0.0,1881.0,497.200000,269.0,502.318385,0.0,...,0,0,0,0,0,0,0,1,0,28


Так как данные в датасете отсортированы и это негативно скажется на обучении модели, то перемешаем их

In [85]:
data = data.sample(frac=1)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
5600,1441347,0,53288,8,5.0,1259.0,286.181818,207.0,281.342114,0.0,...,0,1,0,0,0,0,0,0,1,94
15646,406046,0,31774,42,0.0,755.0,152.530303,96.5,163.678104,0.0,...,1,0,0,0,0,0,0,0,1,15
9503,8035,608,69,32,0.0,50.0,4.191489,1.0,8.278756,0.0,...,0,0,0,0,0,1,0,0,0,0
25583,5792,105,188,32,0.0,20.0,9.454545,12.0,6.624522,0.0,...,0,0,0,0,1,0,0,0,0,0
23140,8580,0,589,44,0.0,4.0,0.437086,0.0,0.810332,0.0,...,0,0,0,0,0,1,0,0,0,0


Отделим значения целевого признака в отдельный массив y

In [86]:
y = data[53].values

Оставшиеся признаки сгруппируем в матрицу Х

In [87]:
X = data.loc[:, 0:52].values

Вычислим средние значения признаков (means) и стандартное отклонение (stds) по каждому столбцу матрицы Х. Сгруппируем значения в соответствующие массивы.

In [88]:
means, stds = [np.mean(X, axis=0), np.std(X, axis=0)]

Если у какого-нибудь признака стандартное отклонение равно 0, значит этот признак является константой и не имеет никакого влияния на целевой признак. Определим такой признак в Х и удалим его из выборки:

In [89]:
np.where(stds==0)

(array([37]),)

In [90]:
X = np.delete(X, 37, axis=1)

Пересчитаем means и stds

In [91]:
means, stds = [np.mean(X, axis=0), np.std(X, axis=0)]

Произведем нормировку признаков. От каждого значения матрицы Х отнимем среднее значение по столбцу, а результат поделим на стандартное отклонение этого столбца.

In [92]:
def normalize_features(X):
    for i in range(0, X.shape[0]):
        for j in range(0, X.shape[1]):
            X[i][j] = (X[i][j] - means[j])/stds[j]
    return X

In [93]:
X = normalize_features(X)

Добавим к матрице X столбец из единиц, чтобы не обрабатывать отдельно коэффициент $w_0$ линейной регрессии

In [94]:
X = np.hstack((X, np.ones((len(X), 1))))


Опишем фунциb вычисления метрик качества. Данные показатели характеризует качество применяемой модели

In [95]:
def mserror(y, y_pred):
    return 1/len(y)*sum((y - y_pred) ** 2)

In [96]:
def rmserror(y, y_pred):
    return np.sqrt(mserror(y, y_pred))

In [97]:
def determination(X, y, w):
    return 1 - (sum((y - np.dot(X, w))**2) / sum((y - np.mean(y, axis=0))**2))

Опишем функцию прогноза для линейной модели, принимающую матрицу Х и вектор весов признаков w, возвращающую вектор прогнозов

In [98]:
def linear_prediction(X, w):
    return np.dot(X, w)


Градиент функционала ошибки вычисляется по формуле:

In [99]:
def gradient_step(X, y, w):
    return  np.dot(2/len(y) * X.T, np.add(linear_prediction(X, w), - y))
    

Градиентный спуск будет производиться в цикле по формуле:
    $$\Large w^t = w^{t-1} - lr*gradient step $$ 
    lr - коэффициент шага (learning rate)

In [100]:
def gradient_descent(X, y, w_init, lr, epoch_num):
    weight_dist = np.inf
    w = w_init
    iter_num = 0
    # Склеим матрицу признаков со столбцом из значений целевого признака
    data = np.concatenate((X, y.reshape(len(y), 1)), axis=1)
    # Основной цикл
    for i in range(epoch_num):
        # Перемешаем данные
        np.random.shuffle(data)
        # Разделим таблицу признаков и целевой признак
        X = np.delete(data, 53, axis=1)
        y = np.delete(data, range(53), axis=1).reshape(1, len(y))[0]
        # Пройдемся по датасету batch-ами
        batch_size = 1000
        for batch_idx in range(int(len(y) / batch_size) + 1):
            start_idx = batch_idx * batch_size
            end_idx = start_idx + batch_size
            batch = X[start_idx:end_idx]
            y_batch = y[start_idx:end_idx]
            # вычисляем градиент
            grad = gradient_step(batch, y_batch, w)
            # обновляем параметры
            w = w - lr * grad
        
    return w

Для кросс-валидации разделим датасет на 5 частей (примерно по 8190 строк в каждой)

In [101]:
NUM_FOLDS = 5

In [102]:
data_split = np.array_split(X, NUM_FOLDS)

In [103]:
target_split = np.array_split(y, NUM_FOLDS)

In [104]:
def cross_validation(features, target, num_folds):
    metrics = []
    weights = []
    for idx in range(num_folds):
        data_train = []
        target_train = []
        for i in range(num_folds):
            if i != idx:
                data_train.append(data_split[i])
                target_train.append(target_split[i])
            else:
                data_test = data_split[i]
                target_test = target_split[i]
        data_train = np.concatenate(data_train, axis=0)
        target_train = np.concatenate(target_train)
        # тренировка модели
        grad_desc_weights = gradient_descent(data_train, target_train, np.zeros(X.shape[1]), lr=0.01, epoch_num=10)
        # расчет метрик
        RMSE_train = rmserror(target_train, np.dot(data_train, grad_desc_weights))
        RMSE_test = rmserror(target_test, np.dot(data_test, grad_desc_weights))
        R2_train = determination(data_train, target_train, grad_desc_weights)
        R2_test = determination(data_test, target_test, grad_desc_weights)
        # объединение результатов
        metrics.append([RMSE_train, RMSE_test, R2_train, R2_test])
        weights.append(grad_desc_weights)
    
    return metrics, weights

In [105]:
metrics, weights = cross_validation(data_split, target_split, NUM_FOLDS)

Расчитаем средние значения и дисперсию по метрикам:

In [106]:
metrics_mean = []
metrics_std = []

for i in range(4):
    result_list = []
    for metric in metrics:
        result_list.append(metric[i])
    metrics_mean.append(np.mean(result_list))
    metrics_std.append(np.std(result_list))


Объединим результаты по метрикам в таблицу:

In [107]:
data_metrics = {
        'fold 1': metrics[0], 
        'fold 2': metrics[1], 
        'fold 3': metrics[2], 
        'fold 4': metrics[3], 
        'fold 5': metrics[4],
        'mean': metrics_mean,
        'std': metrics_std
       }

df_metrics = pd.DataFrame(data_metrics, index=['RMSE train', 'RMSE test', 'R2 train', 'R2 test'])

Расчитаем средние значения и дисперсию по весам признаков:

In [108]:
weights_mean = []
weights_std = []

for i in range(53):
    result_list = []
    for weight_list in weights:
        result_list.append(weight_list[i])
    weights_mean.append(np.mean(result_list))
    weights_std.append(np.std(result_list))


Объединим результаты по весам признаков в таблицу:

In [109]:
data_weights = {
    'fold 1': weights[0], 
    'fold 2': weights[1], 
    'fold 3': weights[2], 
    'fold 4': weights[3], 
    'fold 5': weights[4],
    'mean': weights_mean,
    'std': weights_std
}
indexes = []
for i in range(len(weights[0])):
    indexes.append(f'w{i}')
    
df_weights = pd.DataFrame(data_weights, index=indexes)

Объединим две таблицы в общую таблицу результатов и сохраним ее в 'result.csv':

In [110]:
result_df = pd.concat([df_metrics, df_weights])
result_df.to_csv('result.csv')

result_df

,fold 1,fold 2,fold 3,fold 4,fold 5,mean,std
RMSE train,28.795912,28.447291,29.934452,30.310539,28.859657,29.269570,0.720306
RMSE test,31.805267,32.743375,26.849601,24.916947,31.158273,29.494693,3.053275
R2 train,0.342001,0.314555,0.316002,0.304885,0.320992,0.319687,0.012318
R2 test,0.196137,0.320010,0.319234,0.386747,0.302925,0.305011,0.061589
w0,-1.143819,0.001975,0.501405,0.179844,-0.087126,-0.109544,0.554918
w1,-0.694136,-0.469135,-0.580960,-0.562802,-0.772377,-0.615882,0.106005
w2,-0.105876,-1.776222,-1.491726,-1.524296,-1.364927,-1.252609,0.588689
w3,-0.026995,-0.029952,-0.089148,-0.076605,-0.056191,-0.055778,0.024670
w4,-0.256137,0.399861,-0.414254,0.057571,-0.794032,-0.201398,0.406963
w5,0.418186,0.483811,0.339607,0.376821,0.614707,0.446626,0.096715


Полученные данные по метрикам качества модели объединим в таблицу Results_mini_batch.xls.
Проанализировав полученные результаты, можно сделать вывод: большая часть результатов соответствуют установленным нормам, что говорит об успешном решении поставленной задачи. 